импорты и константы

In [1]:
import os
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
import re
import shutil
import time

# 153.8336486816406250 884.2712384333286764    измеряемый диапазон. 0-2136 диапазон данных
start = 400  # нм
end = 700  # нм
step = (884.2712384333286764 - 153.8336486816406250) / 2136

# номера точек в массиве по координате х
start_point = round((start - 153.8336486816406250) / step)
end_point = start_point + int((end - start) / step)
_start = start_point + 11
_end = start_point + round((end_point - start_point) / 2)

функции получения данных

In [2]:
def get_max_rmr(path, fl):
    _dm = np.zeros(len(fl))

    for i in range(len(fl)):
        with open(path + fl[i], "r", encoding="utf8") as spec:
            _dm[i] = max(re.split(",", spec.read())[_start:_end])
    return _dm

получение максимумов из спектров

In [22]:
data_max = np.zeros(1)  # изанчальные графики
main_folder = r"C:\Users\Nik\Desktop\tes\c2"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
print(main_folder)
file_list = np.array(natsorted(os.listdir(main_folder)))
for i in range(0, len(file_list), 5000):
    _b = i + 5000
    if _b > len(file_list):
        _b = len(file_list)
    data_max = np.append(data_max, get_max_rmr(main_folder, file_list[i:_b]), axis=0)
    print("получено ", _b)
print("получено " + str(len(data_max)) + " максимумов спектров")

C:/Users/Nik/Desktop/tes/c2/
получено  5000
получено  10000
получено  15000
получено  20000
получено  25000
получено  30000
получено  35000
получено  40000
получено  45000
получено  50000
получено  55000
получено  60000
получено  65000
получено  70000
получено  75000
получено  80000
получено  85000
получено  90000
получено  95000
получено  100000
получено  105000
получено  106111
получено 106112 максимумов спектров


максимумы на таймлайнах

In [23]:
_step = 400
_sp = 0
_ep = _step

# создание папки для сохранения картинок
dist_folder = r"C:\Users\Nik\Desktop\prog\1"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"
folder_name = "c2"
if not os.path.exists(os.path.join(dist_folder, folder_name)):
    os.makedirs(os.path.join(dist_folder, folder_name))

# расчет гистограммы
num_bins = 10  # количество интервалов
range_values = (np.max(data_max) - np.min(data_max)) / num_bins
# границы интервалов
bin_edges = np.arange(np.min(data_max), np.max(data_max) + range_values, range_values)
# середины интервалов
bins = bin_edges[:-1] + range_values / 2

hist, _ = np.histogram(data_max, bins=bin_edges)

zero = bin_edges[np.argmax(hist) + 1]
fig, ax = plt.subplots(figsize=(10, 6), dpi=120)
fig.suptitle("гистограмма")
ax.hist(data_max, bin_edges, alpha=0.6, color="blue", edgecolor="k", linewidth=1)
ax.plot(bins, hist, color="red", lw=2)
plt.xticks(rotation=90)
ax.set_xticks(bin_edges)
plt.savefig(dist_folder + "/" + folder_name + "/" + "гистограмма.png")

m = np.min(data_max) * 0.8
m2 = np.max(data_max) * 0.5

for i in range(round(len(data_max) / (3 * _step))):
    fig, axs = plt.subplots(3, 1, figsize=(10, 6), dpi=120)
    plt.ioff()
    _s = _sp
    for j in range(3):
        if _ep > len(data_max):
            _ep = len(data_max)
        axs[j].plot(np.arange(_sp, _ep, 1), data_max[_sp:_ep], lw=1, color="blue")
        axs[j].scatter(np.arange(_sp, _ep, 1), data_max[_sp:_ep], s=4, color="black")
        axs[j].plot((_sp, _ep), (zero, zero), color="orange")
        axs[j].set_ylim(0, np.max(data_max[_sp:_ep]) * 1.1)
        _sp += _step
        _ep += _step

    _e = _ep - _step
    fig.suptitle("спектры " + str(_s) + "-" + str(_e))
    plt.savefig(
        dist_folder
        + "/"
        + folder_name
        + "/"
        + "ряд максимумов "
        + str(_s)
        + "-"
        + str(_e)
        + ".png"
    )

In [19]:
point_index = []
_point = False
k = []
zero = 0.05
s = 10
for i in range(0, len(data_max), s):
    ar = np.mean(data_max[i : i + s])
    if ar > zero:
        k.append(i)
        _point = True
    if ar < zero and _point == True:
        point_index.append((k[0], i))
        k = []
        _point = False
    if _point == True and i >= len(data_max) - s:
        point_index.append((k[0], len(data_max) - 1))
print(len(point_index))
print(point_index)

14
[(10, 160), (430, 700), (710, 940), (970, 1120), (1150, 1180), (1190, 1300), (1310, 1340), (1350, 1450), (1460, 1680), (1700, 1870), (1920, 2050), (2060, 2210), (2690, 3890), (3920, 3928)]


In [36]:
for i in range(len(point_index)):
    print(point_index[i], "   ", point_index[i][1] - point_index[i][0])

[5750, 6875]     1125
[9600, 11250]     1650
[14125, 15810]     1685
[18425, 20200]     1775
[22900, 24680]     1780
[32140, 33400]     1260
[36480, 38000]     1520
[40950, 42575]     1625
[45400, 46820]     1420
[49840, 51550]     1710
[58850, 60500]     1650
[63300, 64900]     1600
[67800, 69400]     1600
[72200, 73750]     1550
[76760, 78150]     1390
[85360, 86880]     1520
[90150, 91450]     1300
[94130, 95600]     1470
[98300, 99650]     1350
[102450, 104000]     1550


In [35]:
point_index = [
[5750,6875],
[9600,11250],
[14125,15810],
[18425,20200],
[22900,24680],

[32140,33400],
[36480,38000],
[40950,42575],
[45400,46820],
[49840,51550],

[58850,60500],
[63300,64900],
[67800,69400],
[72200,73750],
[76760,78150],

[85360,86880],
[90150,91450],
[94130,95600],
[98300,99650],
[102450,104000]]

In [ ]:
p = [
[5750,6875],
[9600,11250],
[14125,15810],
[18425,20200],
[22900,24680],

[32140,33400],
[36480,38000],
[40950,42575],
[45400,46820],
[49840,51550],

[58850,60500],
[63300,64900],
[67800,69400],
[72200,73750],
[76760,78150],

[85360,86880],
[90150,91450],
[94130,95600],
[98300,99650],
[102450,104000],


]

In [37]:
dist_folder = r"C:\Users\Nik\Desktop\tes"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"

folder_name = "с2 points"
d_name = "2_"


if os.path.exists(os.path.join(dist_folder, folder_name)):

    shutil.rmtree(dist_folder + "/" + folder_name)


os.makedirs(os.path.join(dist_folder, folder_name))


for i in range(len(point_index)):

    os.makedirs(os.path.join(dist_folder + "/" + folder_name, d_name + str(i + 1)))

In [38]:
main_folder = r"C:\Users\Nik\Desktop\tes\c2"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
file_list = np.array(natsorted(os.listdir(main_folder)))
print(dist_folder)


for i in range(len(point_index)):

    dist = dist_folder + "/" + folder_name + "/" + d_name + str(i + 1)

    for j in range(point_index[i][0], point_index[i][1]):

        shutil.copy2(main_folder + str(file_list[j]), dist)

    print(d_name, i + 1, " ", point_index[i][1] - point_index[i][0])

C:/Users/Nik/Desktop/tes/
2_ 1   1125
2_ 2   1650
2_ 3   1685
2_ 4   1775
2_ 5   1780
2_ 6   1260
2_ 7   1520
2_ 8   1625
2_ 9   1420
2_ 10   1710
2_ 11   1650
2_ 12   1600
2_ 13   1600
2_ 14   1550
2_ 15   1390
2_ 16   1520
2_ 17   1300
2_ 18   1470
2_ 19   1350
2_ 20   1550
